# 03 — EDA avançado: feature engineering (RFM) + Parquet

Objetivo: criar features úteis para análise e ML e salvar em formatos eficientes.

Tempo: ~25–30 min

## O que você vai fazer

1- Construir features no nível de **cliente** (RFM)  
2- Salvar o dataset final em **Parquet** (melhor para dados)  
3- Ler o Parquet com **pandas**, **pyarrow** e **polars**  

Resultado: `dados/processed/rfm_features.parquet`

In [ ]:
from __future__ import annotations

from pathlib import Path

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

sns.set_theme(style="whitegrid")

def find_repo_root(start: Path | None = None) -> Path:
    """Sobe diretórios até achar README.md + pasta data."""
    cur = (start or Path.cwd()).resolve()
    for _ in range(10):
        if (cur / "README.md").exists() and (cur / "data").exists():
            return cur
        cur = cur.parent
    return Path.cwd().resolve()

root = find_repo_root()
DATA = root / "data"

In [ ]:
sales = pd.read_csv(DATA / "sample" / "sales.csv")
customers = pd.read_csv(DATA / "sample" / "customers.csv")

sales["date"] = pd.to_datetime(sales["date"])
customers["signup_date"] = pd.to_datetime(customers["signup_date"])

df = sales.merge(customers, on="customer_id", how="left")

df.head()

## Feature Engineering: RFM (Recency, Frequency, Monetary)

In [ ]:
# Referência de tempo para recency: "dia seguinte ao último pedido"
as_of = df["date"].max() + pd.Timedelta(days=1)
print("as_of:", as_of.date())

# Agregação por cliente
rfm = (
    df.groupby("customer_id")
      .agg(
          last_purchase=("date", "max"),
          frequency=("order_id", "nunique"),
          monetary=("revenue", "sum"),
          avg_order_value=("revenue", "mean"),
          category_nunique=("category", "nunique"),
          region_nunique=("region", "nunique"),
      )
      .reset_index()
)

rfm["recency_days"] = (as_of - rfm["last_purchase"]).dt.days
rfm = rfm.merge(customers[["customer_id", "segment", "signup_date"]], on="customer_id", how="left")
rfm.head()

## Criando um alvo simples (VIP) para ML depois

In [ ]:
# Define VIP como top 20% por monetary
threshold = rfm["monetary"].quantile(0.80)
rfm["is_vip"] = (rfm["monetary"] >= threshold).astype(int)

rfm[["customer_id", "segment", "recency_days", "frequency", "monetary", "is_vip"]].head()

## Salvando em Parquet (pyarrow)

In [ ]:
out_dir = DATA / "processed"
out_dir.mkdir(parents=True, exist_ok=True)

parquet_path = out_dir / "rfm_features.parquet"
csv_path = out_dir / "rfm_features.csv"

rfm.to_parquet(parquet_path, index=False)  # engine pyarrow
rfm.to_csv(csv_path, index=False)

print("Salvo:", parquet_path)
print("Salvo:", csv_path)

## Lendo com pandas / pyarrow / polars

In [ ]:
import pyarrow.parquet as pq
import polars as pl

rfm_pd = pd.read_parquet(DATA / "processed" / "rfm_features.parquet")
rfm_arrow = pq.read_table(DATA / "processed" / "rfm_features.parquet")
rfm_pl = pl.read_parquet(DATA / "processed" / "rfm_features.parquet")

print("pandas:", rfm_pd.shape)
print("pyarrow:", rfm_arrow.num_rows, rfm_arrow.num_columns)
print("polars:", rfm_pl.shape)

## Exercícios (10–15 min)

1- Crie uma feature `tenure_days` = dias desde `signup_date` até `as_of`.  
2- Crie `monetary_per_day` = monetary / tenure_days (com cuidado com zero).  
3- Faça um gráfico: `recency_days` vs `monetary` (scatter).